# Starter Classes

All neccesary classes and start of project

In [1]:
#All neccesary classes for project

#general
import seaborn as sns
import scipy.stats as ss
import pandas as pd
import numpy as np
import os
import pickle

#for preprocessing
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Imputer
from sklearn.pipeline import FeatureUnion
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import tensorflow as tf

#for machine learning
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import expon, reciprocal
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz

#for evaluation
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

#for utility packages
from Utilities.utilities import import_data
from Utilities.utilities import DataFrameSelector
from Utilities.utilities import CategoricalEncoder
from Utilities.utilities import display_scores
from Utilities.utilities import pipeline_transform
from Utilities.utilities import reset_graph
from Utilities.models import DNN_Model
from Utilities.models import cross_val_score_dnn
from functools import partial

#image manipulation
from PIL import Image
from resizeimage import resizeimage
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from keras.utils.data_utils import get_file
import vgg_preprocessing
import tensorflow.contrib.slim as slim
from tensorflow.contrib.slim.nets import resnet_v2
import pnasnet as nas

Using TensorFlow backend.


# Build NASNET Large

In [ ]:
with open("C:/Users/sdgeo/Dropbox/Own/Programming/Python/MachineLearningDemo/Playground/ml/test_images/dog.png", 'r+b') as f:
    with Image.open(f) as image:
        cover = resizeimage.resize_cover(image, [299, 299])
        cover.save("C:/Users/sdgeo/Dropbox/Own/Programming/Python/MachineLearningDemo/Playground/ml/test_images/dog_shaped.png", image.format)
test_image = mpimg.imread("C:/Users/sdgeo/Dropbox/Own/Programming/Python/MachineLearningDemo/Playground/ml/test_images/dog_shaped.png")[:, :, :3]
plt.imshow(test_image)
plt.axis("off")
plt.show()
test_image = 2 * test_image - 1
X_test = test_image.reshape(1, 299, 299, 3)
X_test = X_test.reshape(1,299,299,3)

In [ ]:
from inception_v4 import create_inception_v4
reset_graph()

def get_file_lists(data_dir):
    import glob
 
    train_list = glob.glob(data_dir + '/' + 'train-*')
    valid_list = glob.glob(data_dir + '/' + 'validation-*')
    if len(train_list) == 0 and \
                    len(valid_list) == 0:
        raise IOError('No files found at specified path!')
    return train_list, valid_list
 
def parse_record(raw_record, is_training):
    """Parse an ImageNet record from `value`."""
    keys_to_features = {
        'image/encoded':
            tf.FixedLenFeature((), tf.string, default_value=''),
        'image/format':
            tf.FixedLenFeature((), tf.string, default_value='jpeg'),
        'image/class/label':
            tf.FixedLenFeature([], dtype=tf.int64, default_value=-1),
        'image/class/text':
            tf.FixedLenFeature([], dtype=tf.string, default_value=''),
    }
 
    parsed = tf.parse_single_example(raw_record, keys_to_features)
 
    image = tf.image.decode_image(
        tf.reshape(parsed['image/encoded'], shape=[]),
        3)
 
    # Note that tf.image.convert_image_dtype scales the image data to [0, 1).
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
 
    image = tf.image.resize_image_with_crop_or_pad(image,299,299)
    
    label = tf.cast(
        tf.reshape(parsed['image/class/label'], shape=[]),
        dtype=tf.int32)
 
    return image, label

def get_batch(is_training, filenames, batch_size, num_epochs=1, num_parallel_calls=1):
    dataset = tf.data.TFRecordDataset(filenames)
 
    if is_training:
        dataset = dataset.shuffle(buffer_size=1500)
 
    dataset = dataset.map(lambda value: parse_record(value, is_training),
                          num_parallel_calls=num_parallel_calls)
    dataset = dataset.shuffle(buffer_size=10000)
    dataset = dataset.batch(batch_size)
    dataset = dataset.repeat(num_epochs)
    iterator = dataset.make_one_shot_iterator()
 
    features, labels = iterator.get_next()
 
    return features, labels

def build_iterator(is_training, filenames, batch_size, num_epochs=1000, num_parallel_calls=12):
    with tf.device("/CPU:0"):
        dataset = tf.data.TFRecordDataset(filenames)

        if is_training:
            dataset = dataset.shuffle(buffer_size=1500)

        dataset = dataset.map(lambda value: parse_record(value, is_training),
                              num_parallel_calls=num_parallel_calls)
        dataset = dataset.shuffle(buffer_size=10000)
        dataset = dataset.batch(batch_size)
        dataset = dataset.repeat(num_epochs)
        iterator = dataset.make_one_shot_iterator()
    return iterator

def get_values(sess, a,b):
    a, b = sess.run([a,b])
    return a,b - 1

In [ ]:
train_list, val_list = get_file_lists("C:/Users/sdgeo/Dropbox/Own/Programming/Inception/new_categories/tf_record")


In [ ]:
reset_graph()

X = tf.placeholder(tf.float32, shape=[None, 299, 299, 3], name="input")
y = tf.placeholder(tf.int32, shape=[None], name="output")
training = tf.placeholder_with_default(False, shape=(), name = 'training')
bn_batch_norm_layer = partial(tf.layers.batch_normalization, training=training, momentum=0.9)
he_init = tf.contrib.layers.variance_scaling_initializer()


dropout_rate = 0.5
learning_rate = 0.01

#Here I need to design something to serve as input in order to do training. 

In [ ]:
n_hidden1 = 500
n_hidden2 = 250
n_hidden3 = 100
n_hidden4 = 50
n_hidden5 = 25
n_hidden6 = 10
n_final_layer = 2


with slim.arg_scope(nas.pnasnet_large_arg_scope()):
    net, end_points = nas.build_pnasnet_large(X, num_classes=1001, is_training=False)
    saver = tf.train.Saver()
    soft_max_pna = tf.get_default_graph().get_tensor_by_name("final_layer/predictions:0")
    last_feature_node = tf.get_default_graph().get_tensor_by_name("final_layer/dropout/Identity:0")

    

with tf.name_scope("DNN_Classifier"):

    stop_layer = tf.stop_gradient(last_feature_node)
    hidden1 = tf.layers.dense(stop_layer, n_hidden1, name="hidden1", kernel_initializer=he_init)
    hidden1_drop = tf.layers.dropout(hidden1, dropout_rate, training=training)
    bn1 = bn_batch_norm_layer(hidden1_drop)
    bn1_act = tf.nn.elu(bn1)
    
    hidden2 = tf.layers.dense(bn1_act, n_hidden2, name="hidden2", kernel_initializer=he_init)
    hidden2_drop = tf.layers.dropout(hidden2, dropout_rate, training=training)
    bn2 = bn_batch_norm_layer(hidden2_drop)
    bn2_act = tf.nn.elu(bn2)
    
    hidden3 = tf.layers.dense(bn2_act, n_hidden3, name="hidden3", kernel_initializer=he_init)
    hidden3_drop = tf.layers.dropout(hidden3, dropout_rate, training=training)
    bn3 = bn_batch_norm_layer(hidden3_drop)
    bn3_act = tf.nn.elu(bn3)
    
    hidden4 = tf.layers.dense(bn3_act, n_hidden4, name="hidden4", kernel_initializer=he_init)
    hidden4_drop = tf.layers.dropout(hidden4, dropout_rate, training=training)
    bn4 = bn_batch_norm_layer(hidden4_drop)
    bn4_act = tf.nn.elu(bn4)
    
    hidden5 = tf.layers.dense(bn4_act, n_hidden5, name="hidden5", kernel_initializer=he_init)
    hidden5_drop = tf.layers.dropout(hidden5, dropout_rate, training=training)
    bn5 = bn_batch_norm_layer(hidden5_drop)
    bn5_act = tf.nn.elu(bn5)
    
    hidden6 = tf.layers.dense(bn5_act, n_hidden6, name="hidden6", kernel_initializer=he_init)
    hidden6_drop = tf.layers.dropout(hidden6, dropout_rate, training=training)
    bn6 = bn_batch_norm_layer(hidden6_drop)
    bn6_act = tf.nn.elu(bn6)
    
    logits_before_bn = tf.layers.dense(bn6_act, n_final_layer, name="outputs")
    logits = bn_batch_norm_layer(logits_before_bn, name="logits")
    softmax = tf.nn.softmax(logits)
    
    with tf.name_scope("loss"):
            xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
            loss = tf.reduce_mean(xentropy, name="loss")

    with tf.name_scope("train"):
        global_step = tf.Variable(0, trainable=False, name='global_step')
        optimizer = tf.train.AdamOptimizer(learning_rate)
        training_op = optimizer.minimize(loss, global_step=global_step)
        

    with tf.name_scope("eval"):
        correct = tf.nn.in_top_k(logits, y, 1)
        accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
        accuracy_summary = tf.summary.scalar('accuracy', accuracy)
            
saver2 = tf.train.Saver()
init = tf.global_variables_initializer()
extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

In [ ]:
filewriter = tf.summary.FileWriter("C:/tmp/Cell_Net/current_graph.ckpt", tf.get_default_graph())

In [ ]:
image_file = get_image('C:/Users/sdgeo/Dropbox/Own/Programming/Inception/ILSVRC/Data/DET/test/ILSVRC2017_test_00000001.jpeg')
image = tf.image.decode_jpeg(tf.reshape(image_file, shape=[]),3)
image = tf.image.convert_image_dtype(image, dtype=tf.float32)

image_add = vgg_preprocessing.preprocess_image(image=image,output_height=299,output_width=299,is_training=False)
image_final = tf.image.resize_image_with_crop_or_pad(image,299,299)

In [ ]:
#Test already exisitng weights for the CNN

with tf.Session() as sess:
    saver.restore(sess, 'C:/Users/sdgeo/Dropbox/Own/Programming/Inception/pnas/model.ckpt')
    imag_val = image_final.eval()
    imag_val = imag_val.reshape(1,299,299,3)
    y_raw = soft_max_pna.eval(feed_dict={X: imag_val})

In [ ]:
with tf.Session() as sess:
    saver.restore(sess, 'C:/Users/sdgeo/Dropbox/Own/Programming/Inception/pnas/model.ckpt')
    imag_vals, labels = get_batch(False, train_list, 5, num_epochs=1, num_parallel_calls=1)
    imag = imag_vals.eval()
    y_raw = soft_max_pna.eval(feed_dict={X: imag})

In [ ]:
un_init_variables = []
with tf.Session() as sess:
    saver.restore(sess, 'C:/Users/sdgeo/Dropbox/Own/Programming/Inception/pnas/model.ckpt')
    un_init = set(sess.run(tf.report_uninitialized_variables()))
    for var in un_init:
        name = var.decode("utf-8")
        variable = [var for var in tf.global_variables() if var.op.name==name][0]
        un_init_variables.append(variable)

init_new_vars_op = tf.initialize_variables(un_init_variables)

In [ ]:
with tf.Session() as sess:
    init.run()
    saver.restore(sess, 'C:/Users/sdgeo/Dropbox/Own/Programming/Inception/pnas/model.ckpt')
    saver2.save(sess, 'C:/tmp/Cell_Net/model_v2.ckpt')

In [ ]:
with tf.Session() as sess:
    saver.restore(sess, 'C:/Users/sdgeo/Dropbox/Own/Programming/Inception/pnas/model.ckpt')
    init_new_vars_op.run()
    saver2.save(sess, 'C:/tmp/model_v2.ckpt')

In [ ]:
#Restore session and train

num_epochs = 100
batch_size = 1

with tf.Session() as sess:
    #restore saver, build iterator, set the step to the global step
    saver2.restore(sess, 'C:/Users/sdgeo/Dropbox/Own/Programming/Inception/model_v2/model_v2.ckpt')
    iterator = build_iterator(True, train_list, batch_size, num_epochs=num_epochs, num_parallel_calls=12)    
    step = tf.train.global_step(sess, global_step)
    next_element = iterator.get_next()
    
    for step in range(num_epochs):
        X_val, y_val = next_element
        X_val, y_val = get_values(sess, X_val, y_val)
        _, _, acc_sum = sess.run([training_op, extra_update_ops, accuracy_summary], feed_dict={X: X_val, y: y_val})
        filewriter.add_summary(acc_sum, step)
        

In [ ]:

num_epochs = 100
batch_size = 1

with tf.Session() as sess:
    #restore saver, build iterator, set the step to the global step
    saver2.restore(sess, 'C:/Users/sdgeo/Dropbox/Own/Programming/Inception/model_v2/model_v2.ckpt')
    iterator = build_iterator(True, train_list, batch_size, num_epochs=num_epochs, num_parallel_calls=12)    
    step = tf.train.global_step(sess, global_step)
    next_element = iterator.get_next()

    X_val, y_val = next_element
    X_val, y_val = get_values(sess, X_val, y_val)


In [ ]:
#Test already exisitng weights for the CNN

with tf.Session() as sess:
    saver2.restore(sess, 'C:/tmp/Cell_Net/model_v2.ckpt')
    
    imag_val = image_final.eval()
    imag_val = imag_val.reshape(1,299,299,3)
    y_raw = soft_max_pna.eval(feed_dict={X: imag_val})

In [ ]:
frame = pd.DataFrame(y_raw)
item = frame.iloc[0,:]
item = np.argmax(item)
item

In [ ]:
pic = Image.fromarray(np.uint8(X_val[0] * 255), 'RGB')

pic

# Using a dataset

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(filenames(False, 'C:/Users/sdgeo/Dropbox/Own/Programming/Inception/ILSVRC/Data/DET/test/'))
dataset = dataset.flat_map(tf.data.TFRecordDataset)
dataset = dataset.map(lambda value: record_parser(value, False),num_parallel_calls=5)
dataset = dataset.prefetch(1)
iterator = dataset.make_one_shot_iterator()
image, label = iterator.get_next()

# Importing a metagraph

In [ ]:
#Making the graph the first time
with tf.Session(config=tf.ConfigProto(allow_soft_placement=True)) as sess:
    new_saver = tf.train.import_meta_graph('C:/Users/sdgeo/Dropbox/Own/Programming/Inception/resnet_v2_checkpoints/model.ckpt-250200.meta')
    new_saver.restore(sess, 'C:/Users/sdgeo/Dropbox/Own/Programming/Inception/resnet_v2_checkpoints/model.ckpt-250200')
    #Check what variables are left uninitialized after loading checkpoint
    uninitialized_variables = set(sess.run(tf.report_uninitialized_variables()))


In [ ]:
#Using the graph after
with tf.Session(config=tf.ConfigProto(allow_soft_placement=True)) as sess:
    saver.restore(sess, "C:/Users/sdgeo/Dropbox/Own/Programming/Inception/resnet_v2_checkpoints/model.ckpt-250200")
    #Code Here

In [ ]:
features